## To merge:
 - Mathew's matlab data
 - my cluster identification data
 - mgefinder
 - breseq (one day, hopefully)


In [ ]:
# add same strain data to awkward array so that each isolate has 
#   1. whether it is a part of a same-strain subset (true/false for filtering)
#   2. the # of SNPs between it and its colleagues
import sys
sys.path.append('PycharmProjects/mge-project/scripts/')
from presnakemake_processing import process_same_strain_file
same_strain_list=process_same_strain_file("~/PycharmProjects/mge-project/workflow/config/All_same_strain_pairs.xlsx")


## Task #1: import all data into person
### Use dask when possible

In [ ]:
import dask.dataframe as dd
import pandas as pd

isolates_list=(pd.read_excel("~/PycharmProjects/mge-project/workflow/config/isolates_list_Maccabi_E.coli_UTI_SeqPlates1to25.xlsx",
     engine="openpyxl")
    .dropna(axis=1,how="all") \
    .dropna(axis=0,how="all") \
    .convert_dtypes(convert_floating=False)
    .SampleDate.apply(str))

In [ ]:

#created nested struct and awkward array from excel
df=isolates_list
ilj = (df.groupby(['NewRandomID','RandomID','CaseNum'])
             .apply(lambda x: x[['IsoNum', 'RowNum','SampleDate','SeqSubDirName','LowQualityThrs', 'Freezer_pos', 'Used', 'SiteString']]
             .to_dict("records"))
             .reset_index()
             .rename(columns={0:'Isolates'})
             .to_json(orient='records'))
#print(json.dumps(json.loads(ilj), indent=2, sort_keys=True))

ila=ak.from_json(ilj)

#add raw seqname to struct
counts=ak.num(ila.Isolates)
Seqname=[str(s).replace('Maccabi_Ecoli_SeqPlate','').replace('Sample_','') for s in ak.flatten(ila.Isolates.SeqSubDirName)]
Seqname=ak.unflatten(Seqname, counts)
ila['Isolates']=ak.with_field(ila.Isolates, Seqname, where="Seqname")

In [ ]:
# process same-strain into json and then awkwardarray
#sd = (same_strain_list.groupby(['RandomID']).aggregate(list)).to_dict()
from presnakemake_processing import process_same_strain_file
same_strain_list=process_same_strain_file("~/PycharmProjects/mge-project/workflow/config/All_same_strain_pairs.xlsx")
ssj = (same_strain_list.groupby('RandomID')['Seqplates']
             .apply(list)
             .reset_index()
             .rename(columns={0:'Isolates'})
             .to_json(orient='records'))

#print(json.dumps(json.loads(ssj), indent=2, sort_keys=True))
ssa=ak.from_json(ssj)

In [ ]:
# merge two arrays in outer join
dictionary, index = np.unique(np.asarray(ssa.RandomID), return_index=True)
closest = np.searchsorted(dictionary, np.asarray(ila.RandomID), side="left")
is_within_range = ak.Array(closest).mask[closest < len(dictionary)]
is_good_match = ak.Array(dictionary)[is_within_range] == ila.RandomID
print(ila.RandomID)
reordering = ak.Array(closest).mask[is_good_match]
ila["Seqplates"] = ssa.Seqplates[index][reordering]


In [ ]:
# Add same-strain using cartesian product
cart=ak.cartesian([ila.Isolates.Seqname, ila.Seqplates], nested=True)
is_ss=ak.any(cart.slot0==cart.slot1, axis=-1)
is_ss=ak.fill_none(is_ss, False)
ila['Isolates']=ak.with_field(ila.Isolates, is_ss, where="IsSameStrain")

## Task #2: import pangenome coverage data
The absolute necessaries:
 - gene names (from roary)
 - gene functions, scaffolds (from prokka)
 - gene coverages (from coverage)
 - BRESEQ
 
All else can wait!
### Work plan
 Load into pangenomes structs and then merge them with dataset at very end

In [ ]:
# get all gene_presence_absence files (older version, works)


# import pyarrow as pa
# from pyarrow import csv
# import pyarrow.parquet as pq

# datadir=os.getcwd()+"/PycharmProjects/mge-project/data/"

# dirnames=np.char.add(np.asarray(datadir+"/pangenome/", dtype=str), np.asarray(ila.NewRandomID,dtype=str))
# filenames=np.char.add(dirnames,np.asarray('/gene_presence_absence.csv', dtype=str))

# isfile=[os.path.isfile(fn) for fn in filenames]

# def ak_string_to_int(akstr):
#     nparr=ak.to_numpy(akstr,allow_missing=True)
#     nparr=np.fromstring(nparr, dtype=int)
#     akint=ak.Array(nparr)
#     return akint


# #get pandas table
# def get_pangenome_data(filename,opt):
#     roary_pa=csv.read_csv(filename, convert_options=opt)
#     roary_pa=roary_pa.rename_columns([n.replace(' ', '_').replace('.','') for n in roary_pa.column_names])
#     roary_pa=roary_pa.drop([col for col in roary_pa.column_names if 'Sample' in col])
#     print(filename)
#     roary_ak=ak.from_arrow(roary_pa)
#     # roary_ak.Accessory_Fragment=ak_string_to_int(roary_ak.Accessory_Fragment)
#     # roary_ak.Accessory_Order_with_Fragment=ak_string_to_int(roary_ak.Accessory_Order_with_Fragment)

#     return roary_ak, ak.num(roary_ak.Gene, axis=0)




# ilp=ila[:4]
# isfile=isfile[:4]
# filenames=filenames[:4]

# opt=csv.ConvertOptions(column_types=[('Accessory Fragment', pa.string()),( 'Accessory Order with Fragment', pa.string() )],
#         strings_can_be_null=True)



# rda=ak.to_arrow(ilp)

# roary_data=[ (get_pangenome_data(fn, opt))  for fn in filenames[isfile] ]

# r_count=np.array([c[1] for c in roary_data])
# count=np.zeros((len(ilp),),dtype=int)
# count[isfile]=r_count

# roary_data=[c[0] for c in roary_data]


# rd=ak.concatenate(roary_data,merge=False, mergebool=False)
# rd=ak.unflatten(rd,count)

# ilp=ak.with_field(ilp, rd, where='Pangenome')
# print(ak.type(ilp))

# ak.to_parquet(ilp, datadir+"/datastruct/up_to_pangenome.parquet")


In [ ]:
# get all gene_presence_absence files (older version, works)


import pyarrow as pa
from pyarrow import csv
import pyarrow.parquet as pq

datadir=os.getcwd()+"/PycharmProjects/mge-project/data/"

dirnames=np.char.add(np.asarray(datadir+"/pangenome/", dtype=str), np.asarray(ila.NewRandomID,dtype=str))
filenames=np.char.add(dirnames,np.asarray('/gene_presence_absence.csv', dtype=str))

isfile=[os.path.isfile(fn) for fn in filenames]

def ak_string_to_int(akstr):
    nparr=ak.to_numpy(akstr,allow_missing=True)
    nparr=np.fromstring(nparr, dtype=int)
    akint=ak.Array(nparr)
    return akint

ilp=ila

roary_pd= [pd.read_csv(fn, error_bad_lines=False) for fn in filenames[isfile]]

count=np.zeros((len(ilp),),dtype=int)
count_r= [len(rpd.index) for rpd in roary_pd]
count[isfile]=count_r

merged_rpd=pd.concat(roary_pd, ignore_index=True)
par=pa.Table.from_pandas(merged_rpd)
rd=ak.from_arrow(par)

rd=ak.unflatten(rd,count)

ilp=ak.with_field(ilp, rd, where='Pangenome')

print(ak.type(ilp))

ak.to_parquet(ilp, datadir+"/datastruct/up_to_pangenome.parquet")